# Day 5: Memory in Langchain"

In [1]:
!pip install langchain
!pip install openai
!pip install pypdf
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
import openai
import langchain
from langchain.prompts import PromptTemplate

from langchain.document_loaders import PyPDFLoader

from langchain.chains import LLMChain

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.llms import OpenAI

import dotenv
dotenv.load_dotenv()

import os

In [5]:
secret_key = os.getenv("OPENAI_API_KEY")

In [7]:
client = OpenAI(openai_api_key = secret_key)